In [20]:
import torch.utils
import torch.utils.data
import torch.utils.data.dataset
import torchvision
import numpy as np
from torch.utils.data import TensorDataset
import torch
from utils.inputdata import InputData
import random
from utils.runner import Runner
import torch.utils
import torch.utils.data
from models.som import SOM
import models.som
import torch
import matplotlib.pyplot as plt
from utils.inputdata import InputData
import argparse
import wandb
import random
from abc import ABC, abstractmethod
from typing import Any
import importlib
importlib.reload(models.som)
from models.som import SOM
from models.stm import STM
import math
import os

In [28]:

alphas = [5, 2, 1]
betas = [0.005, 0.01, 0.02]
api = wandb.Api()
entity = "replearn"
project = "STM-target-radius-experiment-MNIST"
runs = api.runs(
	path=f"{entity}/{project}",
	filters=None,
	order="-created_at"
)
for run in runs:
	#run.delete()
	#print(run.state)
	if run.state!="finished":
		print(run.state)
		#run.delete()



killed


In [47]:
# for i, alpha in enumerate(alphas):
# 	for j, beta in enumerate(betas):
filters = {

	#"state": "finished",
	#"config.MODE": "a",
}
runs = api.runs(
	path=f"{entity}/{project}",
	filters=filters,
	order="-created_at"
)
print(len(runs))
for run in runs:
	print(run.state)
	#if run.state!="finished":
		#run.delete()
		

20
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
